# 付録 10.4.4: Claudeとのビジョン使用に関するベストプラクティス

ビジョンはClaudeとの新しいインタラクションモードを可能にします。画像のパフォーマンスを最大限に引き出すためのいくつかのヒントをまとめました。それに入る前に、まずノートブックを実行するために必要なコードを設定しましょう。

In [ ]:
pip install -qUr requirements.txt

In [ ]:
import boto3
from IPython.display import Image
from botocore.exceptions import ClientError

session = boto3.Session()
region = session.region_name

modelId = 'anthropic.claude-3-sonnet-20240229-v1:0'
#modelId = 'anthropic.claude-3-haiku-20240307-v1:0'

print(f'Using modelId: {modelId}')
print('使用しているリージョン: ', region)

bedrock_client = boto3.client(service_name = 'bedrock-runtime', region_name = region,)

In [ ]:
def get_base64_encoded_image(image_path):
    with open(image_path, "rb") as f:
        image_file = f.read()

    return image_file

## マルチモーダルへの従来の技術の適用

役割割り当てのような従来のプロンプトエンジニアリング技術を使って、幻覚の問題を解決できます。これを例で見てみましょう：

この画像にいる犬の数をClaudeに数えてもらいたいとします。

In [ ]:
Image(filename='./images/best_practices/nine_dogs.jpg')

In [ ]:
messages = [
    {
        "role": 'user',
        "content": [
            {"image": {
                "format": 'jpeg',
                "source": {"bytes": get_base64_encoded_image("./images/best_practices/nine_dogs.jpg")}
                },
            },
            {"text": "この写真には犬が何匹いますか？"},
        ]
    }
]

converse_api_params = {
    "modelId": modelId,
    "messages": messages,
}
response = bedrock_client.converse(**converse_api_params)

# レスポンスから生成されたテキストコンテンツを抽出する
print(response['output']['message']['content'][0]['text'])

犬は9匹しかいないのに、クロードは10匹いると思っています！少しプロンプトエンジニアリングを適用して、もう一度試してみましょう。

In [ ]:
messages = [
    {
        "role": 'user',
        "content": [
            {"image": {
                "format": 'jpeg',
                "source": {"bytes": get_base64_encoded_image("./images/best_practices/nine_dogs.jpg")}
                },
            },
            {"text": "あなたは完璧な視力を持ち、細部に非常に注意を払うため、画像内の物体を数える専門家です。この写真には犬が何匹いますか？ <answer> タグで答えを提供する前に、<thinking> タグで段階的に考え、画像のすべての部分を分析してください。"},
        ]
    }
]

converse_api_params = {
    "modelId": modelId,
    "messages": messages,
}
response = bedrock_client.converse(**converse_api_params)

print(response['output']['message']['content'][0]['text'])

素晴らしい！プロンプトに対していくつかのプロンプトエンジニアリングを適用した後、Claudeは正しく9匹の犬を数えることができるようになりました。

## ビジュアルプロンプティング

画像を入力として使用することで、プロンプトを画像自体の中に与えることが可能になりました。いくつかの例を見てみましょう。

この画像では、テキストを書き、矢印を描いています。これをテキストプロンプトなしでClaudeに渡してみましょう。

In [ ]:
Image(filename='./images/best_practices/circle.png')

In [ ]:
messages = [
    {
        "role": 'user',
        "content": [
            {"image": {
                "format": 'png',
                "source": {"bytes": get_base64_encoded_image("./images/best_practices/circle.png")}
                },
            }
        ]
    }
]

converse_api_params = {
    "modelId": modelId,
    "messages": messages,
    "inferenceConfig": {"maxTokens": 2048},
}
response = bedrock_client.converse(**converse_api_params)

# レスポンスからテキストを出力します
print(response['output']['message']['content'][0]['text'])

ご覧のとおり、Claudeは私たちが質問を与えなかったため、画像を説明しようとしました。画像に質問を追加して、再度渡してみましょう。

In [ ]:
Image(filename='./images/best_practices/labeled_circle.png')

In [ ]:
messages = [
    {
        "role": 'user',
        "content": [
            {"image": {
                "format": 'png',
                "source": {"bytes": get_base64_encoded_image("./images/best_practices/labeled_circle.png")}
                },
            }
        ]
    }
]

converse_api_params = {
    "modelId": modelId,
    "messages": messages,
    "inferenceConfig": {"maxTokens": 2048},
}
response = bedrock_client.converse(**converse_api_params)

print(response['output']['message']['content'][0]['text'])

画像の特定の部分を強調表示し、それについて質問することもできます。

これらの2つの数字の違いは何ですか？

In [ ]:
Image(filename='./images/best_practices/table.png')

In [ ]:
messages = [
    {
        "role": 'user',
        "content": [
            {"image": {
                "format": 'png',
                "source": {"bytes": get_base64_encoded_image("./images/best_practices/table.png")}
                },
            },
            {"text": "これらの二つの数字の違いは何ですか？"},
        ]
    }
]

converse_api_params = {
    "modelId": modelId,
    "messages": messages,
}
response = bedrock_client.converse(**converse_api_params)

# レスポンスから生成されたテキストコンテンツを抽出する
print(response['output']['message']['content'][0]['text'])

## フューショット例

プロンプトに例を追加することで、視覚タスクの精度も向上します。では、Claudeにスピードメーターの画像を読んでもらいましょう。

In [ ]:
Image(filename='./images/best_practices/140.png')

In [ ]:
messages = [
    {
        "role": 'user',
        "content": [
            {"image": {
                "format": 'png',
                "source": {"bytes": get_base64_encoded_image("./images/best_practices/140.png")}
                },
            },
            {"text": "What speed am I going?"},
        ]
    }
]

converse_api_params = {
    "modelId": modelId,
    "messages": messages,
}
response = bedrock_client.converse(**converse_api_params)

# レスポンスから生成されたテキストコンテンツを抽出する
print(response['output']['message']['content'][0]['text'])

クラウドの回答はここではあまり正しく見えません。彼は私たちが時速140キロメートルで走っていると思っていますが、実際には時速140マイルです！もう一度試してみましょう。ただし、今回はプロンプトにいくつかの例を追加しましょう。

In [ ]:
messages = [
    {
        "role": 'user',
        "content": [
            {"image": {
                "format": 'png',
                "source": {"bytes": get_base64_encoded_image("./images/best_practices/70.png")}
                },
            },
            {"text": "What speed am I going?"},
        ]
    },
    {
        "role": 'assistant',
        "content": [
            {"text": "You are going 70 miles per hour."}
        ]
    },
    {
        "role": 'user',
        "content": [
            {"image": {
                "format": 'png',
                "source": {"bytes": get_base64_encoded_image("./images/best_practices/100.png")}
                },
            },
            {"text": "What speed am I going?"},
        ]
    },
    {
        "role": 'assistant',
        "content": [
            {"text": "You are going 100 miles per hour."}
        ]
    },
    {
        "role": 'user',
        "content": [
            {"image": {
                "format": 'png',
                "source": {"bytes": get_base64_encoded_image("./images/best_practices/140.png")}
                },
            },
            {"text": "What speed am I going?"},
        ]
    },
]


converse_api_params = {
    "modelId": modelId,
    "messages": messages,
}
response = bedrock_client.converse(**converse_api_params)

# レスポンスから生成されたテキストコンテンツを抽出する
print(response['output']['message']['content'][0]['text'])

完璧です！その例を通じて、Claudeはスピードメーターの速度を読み取る方法を学びました。ただし、画像を使ったfew-shot promptingは常に機能するわけではありませんが、あなたのユースケースで試してみる価値はあります。

## 複数の画像を入力として使用
Claudeは、プロンプト内で複数の画像を同時に受け入れ、推論することもできます！例えば、非常に大きな画像、例えば長い領収書の画像があるとしましょう！その画像をいくつかのチャンクに分割し、それぞれのチャンクをClaudeに入力することができます。

In [ ]:
Image(filename='./images/best_practices/receipt1.png')

In [ ]:
Image(filename='./images/best_practices/receipt2.png')

In [ ]:
messages = [
    {
        "role": 'user',
        "content": [
            {"image": {"format": 'png',"source": {"bytes": get_base64_encoded_image("./images/best_practices/receipt1.png")}},},
            {"image": {"format": 'png',"source": {"bytes": get_base64_encoded_image("./images/best_practices/receipt2.png")}},},
            {"text": "レストランの名前と合計を出力します。"},
        ]
    }
]

converse_api_params = {
    "modelId": modelId,
    "messages": messages,
}
response = bedrock_client.converse(**converse_api_params)

# レスポンスから生成されたテキストコンテンツを抽出します
print(response['output']['message']['content'][0]['text'])

## 例からの物体識別

画像入力を使用すると、プロンプトに他の画像を渡すことができ、Claudeはその情報を使用して質問に答えます。これを例で見てみましょう。

例えば、画像内のパンツの種類を特定しようとしているとします。プロンプトに異なる種類のパンツのいくつかの例をClaudeに提供することができます。

In [ ]:
Image(filename='./images/best_practices/officer_example.png')

In [ ]:
messages = [
    {
        "role": 'user',
        "content": [
            {"image": {"format": 'png',"source": {"bytes": get_base64_encoded_image("./images/best_practices/wrinkle.png")}},},
            {"image": {"format": 'png',"source": {"bytes": get_base64_encoded_image("./images/best_practices/officer.png")}},},
            {"image": {"format": 'png',"source": {"bytes": get_base64_encoded_image("./images/best_practices/chinos.png")}},},
            {"image": {"format": 'png',"source": {"bytes": get_base64_encoded_image("./images/best_practices/officer_example.png")}},},
            {"text": "これらのパンツは（順番に）しわになりにくいドレスパンツ、イタリアンメルトンオフィサーパンツ、スリムラピッドムーブメントチノです。最後の画像に表示されているパンツは何ですか？"},
        ]
    }
]

converse_api_params = {
    "modelId": modelId,
    "messages": messages,
}
response = bedrock_client.converse(**converse_api_params)

# レスポンスから生成されたテキストコンテンツを抽出します
print(response['output']['message']['content'][0]['text'])